In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade  !! This has been done

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=c3e40cfb58143f1baca7e5f60af88095fb640fce30c1cc9f3cb6719f57a19329
  Stored in directory: c:\users\gshre\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib  !! This has been done

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
print(df.shape)
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
# remove false positive records and other records.....can't do this
#df = df[~(df["koi_disposition"].str.match("CANDIDATE"))]
print(df.shape)
df.head()

(6991, 41)
(6991, 41)


,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
df.koi_disposition.unique()

array(['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

# Select your features (columns)

In [4]:
# Set features. This will also be used as your x values.
df['koi_fpflag_nt'] = df['koi_fpflag_nt'].astype('category')
df['koi_fpflag_ss'] = df['koi_fpflag_ss'].astype('category')
df['koi_fpflag_co'] = df['koi_fpflag_co'].astype('category')
df['koi_fpflag_ec'] = df['koi_fpflag_ec'].astype('category')

In [25]:
# Copy of all features
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
                        'koi_period', 'koi_period_err1', 'koi_period_err2', 
                        'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 
                        'koi_impact', 'koi_impact_err1','koi_impact_err2',
                        'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 
                        'koi_depth', 'koi_depth_err1', 'koi_depth_err2',
                        'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 
                        'koi_teq', 
                        'koi_insol',  'koi_insol_err1', 'koi_insol_err2',
                        'koi_model_snr', 
                        'koi_tce_plnt_num',
                        'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 
                        'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 
                        'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 
                        'ra', 
                        'dec', 
                        'koi_kepmag' 
                       ]]

selected_features.describe()

,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,6991.000000,6.991000e+03,6.991000e+03,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,...,6991.000000,6991.000000,6991.000000,6991.00000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000
mean,56.191248,1.851122e-03,-1.851122e-03,164.488820,0.009340,-0.009340,0.730631,1.945368,-0.337243,5.549368,...,-161.206980,4.305049,0.121091,-0.14048,1.740749,0.352710,-0.388568,292.082406,43.812143,14.271508
std,117.570962,7.184503e-03,7.184503e-03,67.020475,0.021989,0.021989,3.346956,9.401850,1.349036,6.425205,...,71.448481,0.439238,0.132048,0.08199,5.903415,0.839017,1.907797,4.762908,3.606167,1.350802
min,0.259820,1.100000e-08,-1.568000e-01,120.515914,0.000009,-0.569000,0.000000,0.000000,-59.320000,0.167000,...,-1733.000000,0.047000,0.000000,-1.00700,0.109000,0.000000,-103.825000,279.856080,36.577381,6.966000
25%,2.620126,5.005000e-06,-2.401000e-04,132.683917,0.001145,-0.010000,0.198000,0.039900,-0.446000,2.446000,...,-197.000000,4.209000,0.044000,-0.19500,0.829000,0.128000,-0.252000,288.704730,40.797760,13.455000
50%,8.947426,3.300000e-05,-3.300000e-05,136.739230,0.003990,-0.003990,0.539000,0.189800,-0.204000,3.793000,...,-159.000000,4.436000,0.070000,-0.12700,0.999000,0.248000,-0.111000,292.314760,43.679661,14.534000
75%,34.282605,2.401000e-04,-5.005000e-06,169.937005,0.010000,-0.001145,0.891000,0.379000,-0.044000,6.197500,...,-112.000000,4.543000,0.149000,-0.08800,1.357000,0.357000,-0.069000,295.888550,46.693659,15.322000
max,1071.232624,1.568000e-01,-1.100000e-08,1472.522306,0.569000,-0.000009,100.806000,85.540000,0.000000,138.540000,...,0.000000,5.364000,1.472000,0.00000,180.013000,25.956000,0.000000,301.720760,52.336010,19.065000


# Remove err values
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
                        'koi_period', 
                        'koi_time0bk',
                        'koi_impact', 
                        'koi_duration', 
                        'koi_depth',
                        'koi_prad', 
                        'koi_teq', 
                        'koi_insol', 
                        'koi_model_snr', 
                        'koi_tce_plnt_num',
                        'koi_steff', 
                        'koi_slogg',
                        'koi_srad',
                        'ra', 
                        'dec', 
                        'koi_kepmag' 
                       ]]

selected_features.describe()

In [30]:
len(selected_features.columns)

40

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, df["koi_disposition"], random_state=23)

In [7]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
243,0,0,0,0,5.714624,0.000033,-0.000033,132.638660,0.004320,-0.004320,...,-80,4.365,0.132,-0.096,1.013,0.141,-0.141,296.06424,45.976841,15.596
5885,0,1,0,0,27.308150,0.000004,-0.000004,143.504361,0.000113,-0.000113,...,-163,4.574,0.064,-0.096,0.739,0.127,-0.068,291.37350,51.153381,15.529
3093,0,0,0,0,91.957230,0.005680,-0.005680,183.024800,0.065700,-0.065700,...,-79,4.114,0.182,-0.098,1.528,0.238,-0.327,289.90417,38.651371,14.876
2176,0,0,0,0,31.517795,0.000094,-0.000094,185.153300,0.002230,-0.002230,...,-91,4.144,0.195,-0.105,1.400,0.211,-0.316,297.27798,48.320332,12.848
974,0,0,0,0,32.773763,0.000179,-0.000179,151.443520,0.004630,-0.004630,...,-76,4.076,0.203,-0.087,1.534,0.240,-0.360,290.88232,43.471760,14.758


In [8]:
y_train.head()

243          CONFIRMED
5885    FALSE POSITIVE
3093         CANDIDATE
2176         CONFIRMED
974          CANDIDATE
Name: koi_disposition, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:
# Scale your data
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
print(X_train_scaled)

[[-0.43181886 -0.57566178 -0.50696421 ...  0.83425283  0.59986111
   0.97898115]
 [-0.43181886  1.73713112 -0.50696421 ... -0.15216045  2.02573181
   0.9294255 ]
 [-0.43181886 -0.57566178 -0.50696421 ... -0.4611451  -1.41792934
   0.44644283]
 ...
 [-0.43181886 -0.57566178 -0.50696421 ... -1.29672614 -1.29486717
   0.25857514]
 [-0.43181886  1.73713112  1.97252584 ...  1.33190079  0.10835338
  -0.73031895]
 [ 2.31578584 -0.57566178 -0.50696421 ...  1.15163223  0.39505351
   0.89836077]]


In [10]:
# HotKey Encode the data
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
print(encoded_y_train)

[1 2 0 ... 1 2 2]


# Train the Model



In [37]:
# first, create a normal neural network with 40 inputs, 3 hidden nodes, and 3 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=40))
model.add(Dense(units=3, activation='softmax'))

In [38]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 6)                 246       
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 21        
Total params: 267
Trainable params: 267
Non-trainable params: 0
_________________________________________________________________


In [45]:
# Compile the model
model.compile(optimizer='adam',
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])

In [59]:
model.compile(optimizer='adam',
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])

In [60]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    encoded_y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
164/164 - 0s - loss: 1.0188 - accuracy: 0.7038
Epoch 2/100
164/164 - 0s - loss: 0.8349 - accuracy: 0.7385
Epoch 3/100
164/164 - 0s - loss: 0.5744 - accuracy: 0.7831
Epoch 4/100
164/164 - 0s - loss: 0.4712 - accuracy: 0.8152
Epoch 5/100
164/164 - 0s - loss: 0.4118 - accuracy: 0.8375
Epoch 6/100
164/164 - 0s - loss: 0.3730 - accuracy: 0.8472
Epoch 7/100
164/164 - 0s - loss: 0.3479 - accuracy: 0.8516
Epoch 8/100
164/164 - 0s - loss: 0.3311 - accuracy: 0.8558
Epoch 9/100
164/164 - 0s - loss: 0.3182 - accuracy: 0.8621
Epoch 10/100
164/164 - 0s - loss: 0.3085 - accuracy: 0.8669
Epoch 11/100
164/164 - 0s - loss: 0.3016 - accuracy: 0.8695
Epoch 12/100
164/164 - 0s - loss: 0.2959 - accuracy: 0.8737
Epoch 13/100
164/164 - 0s - loss: 0.2911 - accuracy: 0.8743
Epoch 14/100
164/164 - 0s - loss: 0.2873 - accuracy: 0.8756
Epoch 15/100
164/164 - 0s - loss: 0.2838 - accuracy: 0.8798
Epoch 16/100
164/164 - 0s - loss: 0.2810 - accuracy: 0.8833
Epoch 17/100
164/164 - 0s - loss: 0.2784 - accura

In [61]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, encoded_y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.2785 - accuracy: 0.9010
Normal Neural Network - Loss: 0.2785016894340515, Accuracy: 0.9010297656059265


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [44]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [1, 5, 10],
              'max_iter': [1000],
              'n_jobs': [1, 2, 3]}
grid = GridSearchCV(model, param_grid, verbose=3)
print(grid.estimator.get_params().keys())

AttributeError: 'Sequential' object has no attribute 'get_params'

# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [1, 5, 10, 50],
              'kernel': ['linear'],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [26]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] C=1, max_iter=1000, n_jobs=1 ....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ........ C=1, max_iter=1000, n_jobs=1, score=0.860, total=   0.3s
[CV] C=1, max_iter=1000, n_jobs=1 ....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] ........ C=1, max_iter=1000, n_jobs=1, score=0.832, total=   0.3s
[CV] C=1, max_iter=1000, n_jobs=1 ....................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] ........ C=1, max_iter=1000, n_jobs=1, score=0.845, total=   0.3s
[CV] C=1, max_iter=1000, n_jobs=1 ....................................
[CV] ........ C=1, max_iter=1000, n_jobs=1, score=0.826, total=   0.2s
[CV] C=1, max_iter=1000, n_jobs=1 ....................................
[CV] ........ C=1, max_iter=1000, n_jobs=1, score=0.851, total=   0.3s
[CV] C=1, max_iter=1000, n_jobs=2 ....................................
[CV] ........ C=1, max_iter=1000, n_jobs=2, score=0.860, total=   1.0s
[CV] C=1, max_iter=1000, n_jobs=2 ....................................
[CV] ........ C=1, max_iter=1000, n_jobs=2, score=0.832, total=   0.8s
[CV] C=1, max_iter=1000, n_jobs=2 ....................................
[CV] ........ C=1, max_iter=1000, n_jobs=2, score=0.845, total=   0.6s
[CV] C=1, max_iter=1000, n_jobs=2 ....................................
[CV] ........ C=1, max_iter=1000, n_jobs=2, score=0.826, total=   0.6s
[CV] C=1, max_iter=1000, n_jobs=2 ....................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   44.2s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10], 'max_iter': [1000],
                         'n_jobs': [1, 2, 3]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [27]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'max_iter': 1000, 'n_jobs': 1}
0.86705877644285


# Save the Model

In [28]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'logic_regression_model.sav'
joblib.dump(classifier, filename)

['logic_regression_model.sav']